In [19]:
import numpy as np
from astropy.table import Table, vstack

In [20]:
path = './../catalogs/'
VU_GC = Table.read(path+'VUGC_pPXF_2017_v2.fits')
VD_GC = Table.read(path+'VDGC_pPXF_2017_v2.fits')

In [21]:
columns_to_change= ['ZERR', 'ZERR_ne', 'ZERR_pe', 'ZHEL', 'ZOBS', 'ZOBS_ne', 'ZOBS_pe']

for col in columns_to_change:
    VD_GC[col] = VD_GC[col] * 3e+5
    
    VD_GC.rename_column(col,'V' + col[1:])
    
VU_GC['KECKID'] = '---'
VU_GC['ZQUAL'] = -101010
VU_GC['ZSPECNUM'] = -101010

In [32]:
test_VDGC = VD_GC[VD_GC['ZCONF']==1]
test_VDGC = test_VDGC[test_VDGC['GCSAT']== 'Y']
Host = VU_GC[((VU_GC['HOST']== 'VLSB-B') | (VU_GC['HOST']== 'VLSB-D') | (VU_GC['HOST']=='VCC0615')) & (VU_GC['ZCONF']==1) & (VU_GC['GCSAT']=='Y')]
Not_host = VU_GC[((VU_GC['HOST'] != 'VLSB-B') & (VU_GC['HOST'] != 'VLSB-D') & (VU_GC['HOST'] != 'VCC0615') & (VU_GC['ZCONF']==1) & (VU_GC['GCSAT']=='Y'))]
test_VDGC= vstack([test_VDGC, Not_host])
Host.remove_rows(np.nonzero(Not_host)[0])

In [39]:
test_VDGC.remove_rows(np.nonzero((test_VDGC['VHEL']>-300) & (test_VDGC['VHEL'] < 300)))
Host.remove_rows(np.nonzero((Host['VHEL'] > -300) & (Host['VHEL'] < 300)))

In [41]:
Host

VCC,TARGTYPE,GCSAT,HOST,RA,DEC,VHEL,VERR,VERR_pe,VERR_ne,ZCONF,VOBS,VOBS_pe,VOBS_ne,ABANDCOR,ABANDCOR_pe,ABANDCOR_ne,HELCOR,SN,MASKNAME,SLITNUM,YLOW,YHIGH,SPEC1DNAME,KECKID,ZQUAL,ZSPECNUM
bytes10,bytes7,bytes2,bytes10,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes20,int16,int16,int16,bytes40,str3,int64,int64
---,GCXL,Y,VLSB-D,186.17159,13.580055,1006.92,8.453587,4.7,7.19,1.0,987.41,4.7,7.19,-28.51,5.59,6.43,9.0,6.17,vugc5,1,4,7,spec1d.vugc5.001.GCXL001.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.18745,13.564,1057.16,10.3895645,7.36,7.03,1.0,1052.7,7.36,7.03,-13.46,7.44,7.55,9.0,6.33,vugc5,2,24,32,spec1d.vugc5.002.GCXL002.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.17741,13.550445,1026.31,4.880886,3.9,4.49,1.0,1028.8,3.9,4.49,-6.51,2.44,2.55,9.0,11.62,vugc5,3,55,63,spec1d.vugc5.003.GCXL003.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.18413,13.556723,1056.22,12.89066,15.56,8.47,1.0,1056.32,15.56,8.47,-8.9,4.43,4.91,9.0,2.39,vugc5,4,16,22,spec1d.vugc5.004.GCXL004.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.18338,13.483723,1019.73,12.122438,7.14,5.4,1.0,1072.4,7.14,5.4,43.67,13.04,7.71,9.0,5.08,vugc5,6,72,82,spec1d.vugc5.006.GCXL007.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.18124,13.522139,1016.17,2.5402215,2.32,2.21,1.0,1037.49,2.32,2.21,12.32,1.11,1.19,9.0,21.19,vugc5,7,44,52,spec1d.vugc5.007.GCXL009.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.15533,13.511806,1030.47,18.865732,9.73,7.2,1.0,1047.31,9.73,7.2,7.84,16.95,16.77,9.0,4.41,vugc5,10,66,78,spec1d.vugc5.010.GCXL013.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.16821,13.523222,1028.09,3.355,3.79,2.92,1.0,1037.09,3.79,2.92,81.95,76.19,81.31,9.0,8.13,vugc5,11,5,13,spec1d.vugc5.011.GCXL014.fits.gz,---,-101010,-101010
---,GCXL,Y,VLSB-D,186.17363,13.5448885,1057.31,8.705144,8.14,6.45,1.0,1051.42,8.14,6.45,-14.89,5.03,4.47,9.0,6.42,vugc5,13,32,40,spec1d.vugc5.013.GCXL017.fits.gz,---,-101010,-101010
